# Spezialiserung von LLMs mittels LORA
## Am Beispiel deutscher Zitate

### Installation und imports

In [ ]:
!pip install -q torch peft bitsandbytes transformers trl accelerate optimum auto-gptq scipy

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

### Das Grundmodell vorbereiten

In diesem Abschnitt wird das Grundmodell (ein vortrainiertes LLM) heruntergeladen und geladen. Wird ein nicht-quantisiertes Modell geladen, kann dieses für das Training quantisiert in den Speicher geladen werden (auskommentiert).


In [ ]:
base_model_name = "TheBloke/leo-hessianai-7B-GPTQ"
revision = "gptq-8bit-32g-actorder_True" 

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config (if needed)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

In [ ]:
# Das Grundmodell
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    revision=revision,
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

### Den Prompt erstellen

Der Prompt besteht aus 3 Teilen:
* dem System-Prompt, der dem System als Startpunkt gegegeben wird
* dem User-Prompt, der später vom User angegeben wird. Da wir lediglich Zitate von Personen haben wollen, wird auch dieser vorformatiert
* dem Assisstent-Prompt, also der eigentlichen Antwort vom LLM, der im Training aber vorgegeben wird.

Das Format ist durch das Grundmodell vorgegeben und in der jeweiligen Modell-Card auf HuggingFace dokumentiert


In [ ]:
prompt_format = "<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n{target}<|im_end|>\n"
system_prompt = """Dies ist eine Unterhaltung zwischen einem intelligenten, hilfsbereitem KI-Assistenten und einem Nutzer.
Der Assistent gibt Antworten in Form von Zitaten."""

### Die Trainingsdaten vorbereiten
In diesem Abschnitt laden wir die Trainingsdaten in den Arbeitsspeicher und bereiten diese für das Training vor.

In [ ]:
training_data = load_dataset("caretech-owl/wikiquote-de-quotes", split="train")

# Funktion um den kompletten Prompt vorzubereiten
def change_quote(dataitem):
    quote = dataitem["quote"]
    author = dataitem["author"]
    full_prompt = prompt_format.format(system_prompt=system_prompt, prompt="Zitiere " + author, target=quote + " - " + author )
    dataitem.update({"instruct": full_prompt})
    return dataitem
# Datensatz erweitern
training_data = training_data.map(change_quote)

# Text in Token umwandeln, so kann direkt darauf trainiert werden
training_data = training_data.map(lambda samples: tokenizer(samples["instruct"]), batched=True).shuffle()

### Das Lora Modell

Nun definieren wir die LoraConfig. Mittels ```target_modules``` kann man definieren, welche Layer trainiert werden. Der Defaults ist oft ```q_proj, v_proj``` und muss unter Umständen angegeben werden.

In [ ]:
refined_model = "../lora/german_quotes" # User-defined name

peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules=[ 
        #"q_proj",
        #"k_proj",
        #"v_proj",
        #"o_proj",
        #"gate_proj",
        #"up_proj",
        #"down_proj",
        #"lm_head",
    #]
)

### Training

Im folgenden werden Trainingsparameter definiert und das eigentliche Training durchgeführt.


In [ ]:
# Training Params
train_params = TrainingArguments(
    output_dir="../.checkpoints",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    no_cuda=True # comment out for GPU
)

In [ ]:
# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="instruct",
    tokenizer=tokenizer,
    args=train_params,
    max_seq_length=300
)

In [ ]:
# Training (this could take some time
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

### Das Modell testen 
Ohne neu zu laden...


In [ ]:
text_gen = pipeline(task="text-generation", model=fine_tuning.model, tokenizer=llama_tokenizer, max_length=200)
query = prompt_format.format(system_prompt=system_prompt, prompt="Zitiere Oscar Wilde")
output = text_gen(query,do_sample=True, top_p=0.95, max_length=200)
print(output[0]['generated_text'])